#### At first I have manually sorted the files to two different locations and then added them together to create the dataset along with their respective labels.

In [56]:
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import re
import os
import pandas as pd
from sklearn import model_selection, preprocessing,naive_bayes,metrics
from sklearn.feature_extraction.text import TfidfVectorizer
Tokensier = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

In [58]:
def preprocess(file):
    file = re.sub(r'[^a-zA-Z\s]', '', file, re.I|re.A)    # lower case and remove special characters\whitespaces
    file = file.lower()
    file = file.strip()
    tokens = Tokensier.tokenize(file)                                                     
    filtered_tokens = [token for token in tokens if token not in stop_words]
    file = ' '.join(filtered_tokens)
    return file

In [28]:
def create_dataset(typ,norm):
    if typ==1:
        rootdir="/Amen/" # to check define your path for amendment files
        for i in range(1,312):
            with open(rootdir+f'AA{i}.txt', 'r') as myfile:
                data = myfile.read().replace('\n', '')
            data = np.array(data)
            text_mec = np.vectorize(preprocess)
            if i==1:
                norm_corpus = text_mec(data)
            else:
                norm_corpus=np.vstack((norm_corpus,text_mec(data)))
        return norm_corpus
    else:
        root="/Agg_Emp" ## to check define your path for without amendment files
        for subdir, dirs, files in os.walk(root):
            for file in files:
                if file.endswith('.txt'):
                    with open(root+'/'+file, 'r') as myfile:
                        data=myfile.read().replace('\n', '')
                    data = np.array(data)
                    text_mec= np.vectorize(preprocess)
                    norm=np.vstack((norm,text_mec(data)))
        return norm

In [32]:
def Make_Dataframe(norm_corpus):
    df=pd.DataFrame(data=norm_corpus,columns=["text"])
    df['label']=1
    for i,row in df.iterrows():    
        if i>311:
            df.iloc[i,df.columns.get_loc('label')]=0
    result = df.sample(frac=1).reset_index(drop=True)
    return(result)

In [42]:
def train_model(classifier, train_x, train_y, test_x,test_y):
    classifier.fit(train_x, train_y)
    predictions = classifier.predict(test_x)
    return metrics.accuracy_score(predictions, test_y)

In [29]:
norm_corpus=create_dataset(1,[])
norm_corpus=create_dataset(0,norm_corpus)

In [37]:
result=Make_Dataframe(norm_corpus)

In [50]:
train_x,test_x, train_y, test_y = model_selection.train_test_split(result['text'], result['label'])

In [51]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(result['text'])
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(test_x)

In [54]:
tfidf= TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf.fit(result['text'])
train_x_tf =tfidf.transform(train_x)
test_x_tf= tfidf.transform(test_x)

In [55]:
accuracy = train_model(naive_bayes.MultinomialNB(),train_x_tf, train_y,test_x_tf,test_y)
print("Accuracy of Naive Bayes Classifier with TF", accuracy)


Accuracy of Naive Bayes Classifier with TF 0.8924731182795699


In [57]:
accuracy = train_model(naive_bayes.MultinomialNB(fit_prior=False), xtrain_count, train_y, xvalid_count,test_y)
print("Accuracy of Naive Bayes Classifier with Count Vectors", accuracy)

Accuracy of Naive Bayes Classifier with Count Vectors 0.8817204301075269


In [ ]:
## to check the prediction for a file
# Output: 1- Amendment letter 0- employement letter
predictions = classifier.predict(test_x_file)